# EfficientNet-Lite4 - Transferred Learning

---
Link to TensorFlow Hub
https://tfhub.dev/tensorflow/efficientnet/lite4/classification/2

In [28]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
import pathlib


In [66]:
num_classes = 6
# Load the EfficientNet-Lite model from TensorFlow Hub
module_url = "https://tfhub.dev/tensorflow/efficientnet/lite4/classification/2"
model = Sequential([
    hub.KerasLayer(module_url, trainable=False, input_shape=(260, 260, 3)),
    Dense(num_classes, activation='softmax')
])

In [67]:
data_dir = '../SFSU_Art'
data_dir = pathlib.Path(data_dir)
#print(data_dir)

In [68]:
batch_size = 32
img_height = 260
img_width = 260

# Training data  
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

#Validation Data
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

train_classnames = train_ds.class_names
#with open('./models/label.txt', 'w') as f:
#    f.writelines(str(item) + '\n' for item in class_names)



Found 188 files belonging to 6 classes.
Using 151 files for training.
Found 188 files belonging to 6 classes.
Using 37 files for validation.


# Fixing the number output shape

---


In [80]:
# get the configuration of the final layer
output_layer_config = output_layer.get_config()

# print the configuration
print(output_layer_config)

# get the output layer of the model
output_layer = model.layers[-1]


# create a new input layer with 1 unit
new_input_layer = Dense(1)(output_layer.output)

# create a new output layer with the desired number of units and activation function
new_output_layer = Dense(6, activation='softmax')(output_layer.output)

# create a new model with the same input and all layers except the original output layer
new_model = keras.Model(inputs=model.input, outputs=new_output_layer, name='new_model')


{'name': 'dense_10', 'trainable': True, 'dtype': 'float32', 'units': 6, 'activation': 'softmax', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}


In [81]:
# Compile the model with a categorical crossentropy loss function and an Adam optimizer
new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
new_model.summary()


Model: "new_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_5_input (InputL  [(None, 260, 260, 3)]    0         
 ayer)                                                           
                                                                 
 keras_layer_5 (KerasLayer)  (None, 1000)              13118936  
                                                                 
 dense_17 (Dense)            (None, 6)                 6006      
                                                                 
Total params: 13,124,942
Trainable params: 6,006
Non-trainable params: 13,118,936
_________________________________________________________________


In [83]:
# print the configuration of the previous layer
previous_layer_config = model.layers[-2].get_config()
print(previous_layer_config)


{'name': 'keras_layer_5', 'trainable': False, 'dtype': 'float32', 'batch_input_shape': (None, 260, 260, 3), 'handle': 'https://tfhub.dev/tensorflow/efficientnet/lite4/classification/2', 'signature': 'default', 'output_key': 'default'}


In [82]:
# Train the model on a new dataset
epochs = 15
history = new_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/15


ValueError: in user code:

    File "/usr/local/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/usr/local/lib/pyt